# NSMC 감성분석 data set
- data load 및 generation
- 간단한 EDA 와 ml 모델 돌리기
- Feature Engineering 시에는 다양한 노트북 참고 예정  
- Updated 1014


## result list
|data set|corpus|model|accuracy|비고|
|------|---|---|---|---|
|141000개|corpus 초기 494|NB|0.8129517728879261|valid test|
|141000개|corpus 초기 494|DT|0.6820430965682363|valid test|
|141000개|corpus 초기 494|KNN|801527761942766|valid test|
|corpus로 거름|500개|NB|0.8127225013731514|test set, emb만 사용|
|corpus로 거름|500개|DT|0.6799031674024045|test set, emb만 사용|
|corpus로 거름|500개|NB|0.812498728563582|test set, emb300|
|corpus로 거름|500개|DT|0.6684093821836158|test set, emb300|
|corpus로 거름|500개|DT|0.7213011371727323|test set, emb50|

|corpus로 거름|10000개||

0.7213011371727323



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/My Drive/AI_assignment/hackathon_1

/content/drive/My Drive/AI_assignment/hackathon_1


In [3]:
# ready
import pandas as pd
import numpy as np

import sys
np.set_printoptions(threshold=sys.maxsize)
import os
import tqdm
import warnings
warnings.filterwarnings(action='ignore')
import pickle
import joblib

from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

import re

# visualization
from matplotlib import pyplot as plt
plt.style.use('seaborn')
import seaborn as sns
%matplotlib inline

In [4]:
# Font
import matplotlib.font_manager as fm
fm._rebuild()
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

In [5]:
# kras
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

# sklearn
from sklearn.model_selection import train_test_split,KFold
#from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import BaggingClassifier, VotingClassifier

In [6]:
from gensim.models import Word2Vec

### 1) Load Data
데이터 형식  
ID \t document \t label (0:neg / 1:pos) 로 공지 

token <- 한글, duplicate, nan 날림, 형용사 명사 부사 남김, 불용어뺌

In [14]:
# train by hyun
# train_token= joblib.load(os.path.join('data','train_tokens.pickle'))
# train_label= joblib.load(os.path.join('data','train_label.pickle'))

# train by moon
train_token= joblib.load(os.path.join('data','train_moong.pickle'))
train_label= joblib.load(os.path.join('data','train_label.pickle'))

# test 
test_token = joblib.load(os.path.join('data','test_moong.pickle'))
test_label = joblib.load(os.path.join('data','test_label_moong.pickle'))


train 데이터 갯수 : 146182  
test 데이터 갯수 : 49157  
corpus 단어 갯수 : tfidf_vocab500, tfidf_vocab10000


### 2) Word2Vec


In [ ]:
# setting
token = train_token
dim = 128

emb_model = Word2Vec(token, size=dim, window=3, min_count=1, iter=100, sg=1)

# save
emb_model.save('model/word2vec128_train.model')

# # load model
# # emb_model = joblib.load(os.path.join('model','word2vec_train_1015.model'))
# emb_model = Word2Vec.load('model/word2vec128_train_.model')


# 12분
def make_w2v_mat(word_token_list, dim):
    # word dict
    word_table = {
        word : vec for word, vec in zip(
            emb_model.wv.index2word,
            np.array(emb_model.wv.syn0)
        )
    }

    emb_mat = np.zeros((len(word_token_list),dim))

    for i,morphs in enumerate(word_token_list):
        vector = np.array([
                    word_table[morph] for morph in morphs 
                    if morph in word_table
                ])
        if vector.size != 0:
            final_vector = np.mean(vector,axis=0)
            emb_mat[i] = final_vector
    
    return emb_mat

In [9]:
# Data generation
train_emb_mat = make_w2v_mat(train_token,dim)
test_emb_mat = make_w2v_mat(test_token,dim)

print(train_emb_mat.shape, test_emb_mat.shape)


# setting
X_train, X_test, y_train, y_test = train_emb_mat, test_emb_mat, train_label, test_label 

(145791, 128) (49157, 128)


### 3) Modeling

In [10]:
# ml model
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()
nb = GaussianNB()
svm = SVC()

In [11]:
def model_eval(model,X_train, X_test, y_train, y_test, save_model_name):
    
    #X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state=42)
    
    clf = model.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test,y_pred)
    print(model,' accuracy : ', acc)

    # save model
    with open('model/train_' + save_model_name + '.pickle', 'wb') as f:
        pickle.dump(model, f, pickle.HIGHEST_PROTOCOL)
    
    return acc

In [12]:
# SVM 빨리 돌리는 코드

from sklearn.svm import SVC

estimator = nb

n_estimators = 10
n_jobs = -1
model = BaggingClassifier(base_estimator=estimator,
                          n_estimators=n_estimators,
                          max_samples=0.1 ,max_features=1.0,
                          n_jobs=n_jobs)

model.fit(X_train,y_train)
accuracy_score(model.predict(X_test),y_test)

# svm 0.7672762780478873
# knn 0.763370425371768 8분
# dt 0.7054539536586855 7초
# nb 0.7519986980491079

0.7290924995422828

In [13]:
# embedding 128 차원
# NB
print(model_eval(nb,X_train, X_test, y_train, y_test, 'emb128_nb'))
print('-'*50)

# dt
%time print(model_eval(dt,X_train, X_test, y_train, y_test, 'emb128_dt'))
print('-'*50)

# knn
# %time print(model_eval(knn,X_emb, X_test_emb, y_train, y_test, 'model/emb128_knn_voca500'))
# print('-'*50)

GaussianNB(priors=None, var_smoothing=1e-09)  accuracy :  0.7306385662265802
0.7306385662265802
--------------------------------------------------
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')  accuracy :  0.6427161950485181
0.6427161950485181
CPU times: user 41.5 s, sys: 7.93 ms, total: 41.6 s
Wall time: 42.7 s
--------------------------------------------------


## Ensemble Voting

In [ ]:
def Voting (X,y, model1, model2, model3=None, model4=None, model5=None):
    eclf = VotinClassifier(estimators=[
                                       ('model1', model1),
                                       ('model2', model2),
                                       ('model3', model3),
                                       ('model4', model4),
                                       ('model5', model5)
                                       ], voting='soft')
    eclf = eclf.fit(X,y)
    return eclf

In [ ]:
# Ensemble & Evaluate
eclf = Voting(X_train, y_train, nb_w2v, dt_w2v, svm_bag_wvv)
y_pred = eclf.predict(X_test)

accuracy_score(y_test,y_pred)